In [95]:
import torch
data = torch.randn([1, 25088])

In [101]:
x1 = torch.concat([data, torch.zeros([1, 512])], dim=1)
x2 = data

y1 = torch.stft(x1, 512, hop_length=256, return_complex=True)
y2 = torch.stft(x2, 512, hop_length=256, return_complex=True)
y_ = torch.stft(torch.concat([x2[:, 24832:], torch.zeros([1, 256])], dim=1), 512, hop_length=256, return_complex=True, center=False)
y2 = torch.concat([y2[:,:, :-1], y_, torch.zeros([1, 257, y1.shape[2]-y2.shape[2]])], dim=2)
# y2 = torch.concat([y2, torch.zeros([1, 257, y1.shape[2]-y2.shape[2]])], dim=2)

In [102]:
print(torch.sum(torch.abs(torch.abs(y1[:,:, 99]) - torch.abs(y2[:,:, 99]))))

tensor(0.)


In [103]:
print(torch.sum(torch.abs(torch.abs(y1) - torch.abs(y2))))

tensor(0.)


In [104]:
x1 = torch.concat([data, torch.zeros([1, 512])], dim=1)
x2 = data

y1 = torch.stft(x1, 512, hop_length=256, return_complex=True, center=False)
y2 = torch.stft(x2, 512, hop_length=256, return_complex=True, center=False)
y_ = torch.stft(torch.concat([x2[:, 24832:], torch.zeros([1, 256])], dim=1), 512, hop_length=256, return_complex=True, center=False)
y2 = torch.concat([y2, y_, torch.zeros([1, 257, y1.shape[2]-y2.shape[2]-1])], dim=2)
print(torch.sum(torch.abs(torch.abs(y1) - torch.abs(y2))))

tensor(0.)


In [11]:
import time
from progressbar import ProgressBar, Percentage, Bar, Timer, ETA

widgets = ['Progress: ',Percentage(), ' ', Bar('#'),' ', Timer(),
           ' ', ETA()]
progress = ProgressBar(widgets=widgets).start()
for i in progress(range(100)):
    time.sleep(0.04)